In [1]:
from pathlib import Path
import json
import pandas as pd
from sklearn.model_selection import train_test_split


In [4]:
REPO_URL = "https://github.com/eugeniavd/magic_tagger.git"  # <-- EDIT if needed
!git clone {REPO_URL}


Cloning into 'magic_tagger'...
remote: Enumerating objects: 378, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 378 (delta 49), reused 160 (delta 47), pack-reused 205 (from 1)
Receiving objects: 100% (378/378), 14.95 MiB | 17.07 MiB/s, done.
Resolving deltas: 100% (121/121), done.


In [6]:
PROJECT_ROOT = Path("/content/magic_tagger")

csv_path = PROJECT_ROOT / "data" / "processed" / "classify_data_normalized.csv"

# --- load ---
df = pd.read_csv(csv_path, encoding="utf-8")
print("Loaded:", csv_path)
print("Shape:", df.shape)
display(df.head(5))

Loaded: /content/magic_tagger/data/processed/classify_data_normalized.csv
Shape: (50, 14)


,tale_id,rights_status,content_description,set,sampling_version,type_count,collection,volume_no,source_ref,atu_labels_json,txt_path,text_raw,summary_norm,text_norm
0,era_vene_1_503_1,open,[Царевна-лягушка].,core,v1_20251230,3,"ERA, Vene",1,"ERA, Vene 1, 503/4 (1)","[""402""]",/Users/eugenia/Desktop/thesis/magic_tagger/dat...,Тили были царь с царицей у не\nбыло три сына. ...,царевна-лягушка.,тили были царь с царицей у не было три сына. ц...
1,era_vene_1_515_1,open,"[По пьяни мужик спорит, что сможет принести но...",coverage,v1_20251230,1,"ERA, Vene",1,"ERA, Vene 1, 515/6 (1)","[""410""]",/Users/eugenia/Desktop/thesis/magic_tagger/dat...,"Раз пяное, ребятище» подился.\nчто можит в 12 ...","по пьяни мужик спорит, что сможет принести ноч...","раз пяное, ребятище» подился. что можит в 12 ч..."
2,era_vene_12_105_22,open,Снегурочка.,core,v1_20251230,3,"ERA, Vene",12,"ERA, Vene 12, 105 (22)","[""703*""]",/Users/eugenia/Desktop/thesis/magic_tagger/dat...,Сделали дети со снегу куклу.\nВ одного старина...,снегурочка.,сделали дети со снегу куклу. в одного старина ...
3,era_vene_12_137_98,open,Иван-дурак.,core,v1_20251230,4,"ERA, Vene",12,"ERA, Vene 12, 137/41 (98)","[""530""]",/Users/eugenia/Desktop/thesis/magic_tagger/dat...,"Кил-был стажк. В яво бло\nтра сегна. Миша, Гри...",иван-дурак.,"кил-был стажк. в яво бло тра сегна. миша, гриш..."
4,era_vene_12_189_1,open,Два брата.,core,v1_20251230,2,"ERA, Vene",12,"ERA, Vene 12, 189/94 (1)","[""735A""]",/Users/eugenia/Desktop/thesis/magic_tagger/dat...,Жили – брели два брата.\nи посла смерти отца о...,два брата.,жили — брели два брата. и посла смерти отца об...


In [10]:
col = "atu_labels_json"

def parse_labels(x):
    if pd.isna(x):
        return []
    s = str(x).strip()
    if not s:
        return []
    try:
        v = json.loads(s)
        if isinstance(v, list):
            return [str(t).strip() for t in v if str(t).strip()]

        return [str(v).strip()]
    except Exception:

        return [t.strip() for t in s.split(",") if t.strip()]

df["labels"] = df[col].apply(parse_labels)

unique_labels = sorted({lab for labs in df["labels"] for lab in labs})
print("Unique labels:", len(unique_labels))
print("Example:", unique_labels[:20])


Unique labels: 37
Example: ['1000', '1060', '1168', '1174', '300', '300A', '301', '302C*', '302С*', '307', '313', '325', '327A', '331', '402', '410', '425C', '470', '480A', '480D*']


In [12]:
label_counts = pd.Series([lab for labs in df["labels"] for lab in labs]).value_counts()
display(label_counts)

,count
707,6
480D*,5
530,3
703*,3
402,3
552,3
480A,3
650A,3
307,3
301,2
